In [4]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import scipy.stats as stats
import sqlite3

Loading the dataset

In [6]:
#creating the connection

conn = sqlite3.connect('inventory.db')


#fetching vendor summary data
df = pd.read_sql_query("SELECT * FROM vendor_sales_summary", conn)
df.head()

,VendorNumber,VendorName,Brand,Description,PurchasePrice,Volume,TotalPurchaseQuantity,TotalPurchaseDollars,TotalSalesQuantity,TotalSalesDollars,TotalExciseTax,TotalSalesPrice,FreightCost
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,1750.0,145080,3811251.60,142049.0,5.101920e+06,260999.20,672819.31,68601.68
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,1750.0,164038,3804041.22,160247.0,4.819073e+06,294438.66,561512.37,144929.24
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,1750.0,187407,3418303.68,187140.0,4.538121e+06,343854.07,461140.15,123780.22
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,1750.0,201682,3261197.94,200412.0,4.475973e+06,368242.80,420050.01,257032.07
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,1750.0,138109,3023206.01,135838.0,4.223108e+06,249587.83,545778.28,257032.07


In [8]:
#summary statistics

df.describe().T

,count,mean,std,min,25%,50%,75%,max
VendorNumber,10692.0,10650.649458,18753.519148,2.00,3951.0000,7153.000,9552.0000,2.013590e+05
Brand,10692.0,18039.228769,12662.187074,58.00,5793.5000,18761.500,25514.2500,9.063100e+04
PurchasePrice,10692.0,24.385303,109.269375,0.36,6.8400,10.455,19.4825,5.681810e+03
Volume,10692.0,847.360550,664.309212,50.00,750.0000,750.000,750.0000,2.000000e+04
TotalPurchaseQuantity,10692.0,3140.886831,11095.086769,1.00,36.0000,262.000,1975.7500,3.376600e+05
TotalPurchaseDollars,10692.0,30106.693372,123067.799627,0.71,453.4575,3655.465,20738.2450,3.811252e+06
TotalSalesQuantity,10692.0,3077.482136,10952.851391,0.00,33.0000,261.000,1929.2500,3.349390e+05
TotalSalesDollars,10692.0,42239.074419,167655.265984,0.00,729.2200,5298.045,28396.9150,5.101920e+06
TotalExciseTax,10692.0,1774.226259,10975.582240,0.00,4.8000,46.570,418.6500,3.682428e+05
TotalSalesPrice,10692.0,18793.783627,44952.773386,0.00,289.7100,2857.800,16059.5625,6.728193e+05
